In [6]:
import pandas as pd
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.ar_model import AutoReg
from pymongo import MongoClient
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error
from glob import glob

### The goal is to figure out if time series model such as ARIMA or AutoReg will be better predictive models of Air Quality (PM2.5) in Lagos/Nigeria than a regular Linear or Ridge regression

So imagine, you got like 5 csv files from a sleek looking website. Turns out, all the data looking dirty like coz they were processes on MS Excel on a silly MAC

In [9]:
data_files = glob("./data/tmp*.csv")

In [36]:
data_files[0]
pd.read_csv(data_files[0]).head(2)

,sensor_id;sensor_type;location;lat;lon;timestamp;value_type;value
0,4852;DHT22;3627;6.515;3.400;2024-02-01T12:02:5...
1,4852;DHT22;3627;6.515;3.400;2024-02-01T12:02:5...


In [33]:
temp_df = pd.read_csv(data_files[0])
columns = temp_df.columns.str.split(";")
temp_df = temp_df["sensor_id;sensor_type;location;lat;lon;timestamp;value_type;value"].str.split(";", expand=True)
temp_df.columns = list(columns)[0]
temp_df.head()

,sensor_id,sensor_type,location,lat,lon,timestamp,value_type,value
0,4852,DHT22,3627,6.515,3.400,2024-02-01T12:02:52.744148+00:00,humidity,31.00
1,4852,DHT22,3627,6.515,3.400,2024-02-01T12:02:52.744148+00:00,temperature,30.00
2,4852,DHT22,3627,6.515,3.400,2024-02-01T12:03:47.839650+00:00,humidity,31.30
3,4852,DHT22,3627,6.515,3.400,2024-02-01T12:03:47.839650+00:00,temperature,30.10
4,4852,DHT22,3627,6.515,3.400,2024-02-01T12:04:46.909811+00:00,humidity,31.40


In [34]:
temp_df.value_type.value_counts()

value_type
P2             4270
P1             4270
P0             4270
humidity       4144
temperature    4143
Name: count, dtype: int64

In [ ]:
Starting off with the Linear and Ridge Regression Models
